In [545]:
import numpy as np
import time
import pandas as pd
from scipy.stats import qmc, norm
import math
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format

In [235]:
S0 = 100
K = 100
r = 0.05
sigma = 0.1

In [409]:
T = 1
N = 256
dt = T / N
sqrt_dt = np.sqrt(dt)

### Control Variates

In [515]:
def geom_control_function(S0, r, sigma, T, N):
    
    n_pil = 100
    np.random.seed(1)

    res_control_pilot = np.zeros(n_pil)
    res_sample_pilot = np.zeros(n_pil)

    for i in range(n_pil):
        S = np.zeros(N + 1)
        S[0] = S0

        for j in range(1, N + 1):
            S[j] = S[j-1] * math.exp((r - (sigma*sigma)/2) * dt + 
                                     sigma * math.sqrt(dt) * np.random.normal(0, 1))

        S = np.delete(S, 0)

        geo_avg = np.exp(np.log(S).mean())
        arith_avg = S.mean()

        res_control_pilot[i] = np.exp(-1 * r * T) * max(geo_avg - K, 0)
        res_sample_pilot[i] = np.exp(-1 * r * T) * max(arith_avg - K, 0)  
    
    a = np.log(S0) + (r - (sigma**2 / 2)) * T * (N + 1) / (2*N)
    b = (sigma**2) * (T * (N+1) * (2*N + 1)) / (6 * N*N)

    d1 = (-1 * np.log(K) + a + b) / math.sqrt(b)
    d2 = d1 - math.sqrt(b)

    phi_d1 = norm.cdf(d1)
    phi_d2 = norm.cdf(d2)

    mu_g_analytic = np.exp(-1 * r * T) * (np.exp(a + b/2)*phi_d1 - K * phi_d2)
    beta_estimate = np.cov(res_control_pilot, res_sample_pilot)[0][1] / np.var(res_control_pilot)
    
    return mu_g_analytic, beta_estimate

def european_control_function(S0, r, sigma, T, N):
    
    def black_scholes_call(S0, K, T, r, sigma):
        # Calculate d1 and d2
        d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)

        # Calculate the call option price using the Black-Scholes formula
        call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

        return call_price
    
    n_pil = 1000
    np.random.seed(1)

    res_control_pilot = np.zeros(n_pil)
    res_sample_pilot = np.zeros(n_pil)

    for i in range(n_pil):
        S = np.zeros(N + 1)
        S[0] = S0

        for j in range(1, N + 1):
            S[j] = S[j-1] * math.exp((r - (sigma*sigma)/2) * dt + 
                                     sigma * math.sqrt(dt) * np.random.normal(0, 1))

        S = np.delete(S, 0)

        #geo_avg = np.exp(np.log(S).mean())
        euro = S[-1]
        arith_avg = S.mean()

        res_control_pilot[i] = np.exp(-1 * r * T) * max(euro - K, 0)
        res_sample_pilot[i] = np.exp(-1 * r * T) * max(arith_avg - K, 0)  
        
        mu_g_analytic = black_scholes_call(S0, K, T, r, sigma)
        beta_estimate = np.cov(res_control_pilot, res_sample_pilot)[0][1] / np.var(res_control_pilot)

        return mu_g_analytic, beta_estimate
    

def control_variates(n):
    
    mu_g_analytic, beta_estimate = geom_control_function(S0, r, sigma, T, N)
    
    result = np.zeros(n)
    result_control = np.zeros(n)
    for i in range(n):
        S = np.zeros(N + 1)
        S[0] = S0

        for j in range(1, N + 1):
            S[j] = S[j-1] * math.exp((r - (sigma*sigma)/2) * dt + 
                                     sigma * math.sqrt(dt) * np.random.normal(0, 1))

        S = np.delete(S, 0)

        geo_avg = np.exp(np.log(S).mean())
        arith_avg = S.mean()

        control = np.exp(-1 * r * T) * max(geo_avg - K, 0)
        result[i] = np.exp(-1 * r * T) * max(arith_avg - K, 0)
        result_control[i] = result[i] + beta_estimate * (mu_g_analytic - control)
        
    return result_control

In [237]:
def payoff_european(S, call = True):
    if call:
        return np.maximum(S[-1] - K, 0)
    else:
        return np.maximum(K - S[-1], 0)

def payoff_asian(S, call = True):
    if call:
        return np.maximum(S.mean(axis = 0) - K, 0)
    else:
        return np.maximum(K - S.mean(axis = 0), 0)
    
def payoff_asian_importance(S, likelihood_ratio = 0.5, call=True):
    raw_payoff = np.maximum(S.mean(axis=0) - K, 0) if call else np.maximum(K - S.mean(axis=0), 0)
    return raw_payoff * likelihood_ratio

In [333]:
def control_variate_asian_asset(S_i, n):
    Z = np.random.normal(size=n)
    S_new = S_i * (1 + r * dt + sigma * sqrt_dt * Z)
    return S_new

def payoff_asian_cv(S):
    """
    Returns control variate adjusted payoff.
    S: shape (N, M) array of asset prices
    """
    N, M = S.shape
    avg_price = S.mean(axis=1)
    V = np.exp(-r * T) * np.maximum(avg_price - K, 0)

    S_sum = S.sum(axis=1)
    S_star = np.mean(S_sum)

    cov = np.cov(V, S_sum, bias=True)[0, 1]
    var = np.var(S_sum)
    alpha = cov / var if var > 0 else 0.0

    V_cv = V - alpha * (S_sum - S_star)
    return V_cv


In [327]:
def quasi_brownian_asset(S_i, n):
    Z = quasi_random_normal_samples(n)
    return S_i * (1 + r * dt + sigma * sqrt_dt * Z)

def brownian_asset(S_i, n):
    Z = np.random.normal(loc=0, scale = 1, size = n)    
    return S_i * (1 + r * dt + sigma * sqrt_dt * Z)

def antithetic_brownian_asset(S_i, n):
    Z1 = np.random.normal(size = (int) (n/2))
    Z2 = -Z1
    
    Z = np.empty((Z1.size + Z2.size,), dtype=Z1.dtype)
    Z[0::2] = Z1
    Z[1::2] = Z2
    
    return S_i * (1 + r * dt + sigma * sqrt_dt * Z)

def brownian_asset_importance_sampling(S_i, n, mu_shift=0.2):
    Z = np.random.normal(loc=mu_shift, size=n)
    S_path = S_i * (1 + r * dt + sigma * sqrt_dt * Z)
    
    # Likelihood ratio for each Z sample
    likelihood_ratio = np.exp(-mu_shift * Z + 0.5 * mu_shift**2)
    
    return S_path, likelihood_ratio

In [569]:
def quasi_Sobol(n, dim):
    sampler = qmc.Sobol(d=dim, scramble=True)
    u = sampler.random(n)
    z = norm.ppf(u)
    return z

def standard_normal(N, M):
    return np.random.normal(loc = 0, scale = 1.0, size = (N, M))
    

def GBM(S, r, sigma, dt, sqrt_dt, Z):
    return S * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt_dt * Z)

def antithetic_GBM(S, r, sigma, dt, sqrt_dt, Z):
    return S * np.exp((r - 0.5 * sigma**2) * dt - sigma * sqrt_dt * Z)

def gbm_faster(S, r, sigma, dt, sqrt_dt):
    return S * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt_dt * np.random.normal())


def run_monte_carlo(S0, K, r, sigma, T, M, N, payoff_function, method = GBM, sample_method = standard_normal, antithetic = True):
    
    dt = T / M
    sqrt_dt = np.sqrt(dt)
    
    dimension = 1
    
    if antithetic:
        dimension = 2
    
    payoffs = np.zeros( (int) (dimension * N) )
    
    if sample_method == quasi_Sobol:
        Z = sample_method(N, M)
    
    for i in range(N):
        S = S0
        path = [S]
        for j in range(M):
            
            if sample_method == quasi_Sobol:
                S = method(S, r, sigma, dt, sqrt_dt, Z[i, j])
            else:
                S = method(S, r, sigma, dt, sqrt_dt, np.random.normal())
                
            #S = S * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt_dt * Z[i, j])
            path.append(S)

        
        payoffs[i] = payoff_function(np.array(path))

    
    if antithetic:
        
        for i in range(N):
            S = S0
            path = [S]
            for j in range(M):
                
                if sample_method == quasi_Sobol:
                    S = method(S, r, sigma, dt, sqrt_dt, Z[i, j])
                else:
                    S = method(S, r, sigma, dt, sqrt_dt, np.random.normal())
                    
                #S = S * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt_dt * Z[i, j])
                path.append(S)


            payoffs[i + N] = payoff_function(np.array(path))
    
        
    price = np.exp(-r * T) * payoffs
    return np.mean(price), np.std(price) / np.sqrt(dimension * N)


def run_experiment(method = GBM, payoff_function = payoff_asian, num_trials = 10, base = 2, sample_method = standard_normal, antithetic = False, cv = False):
       
    min_samples = 3
        
    n_list = np.arange(min_samples, num_trials, 1)
    v = np.zeros_like(n_list, dtype = np.float64)
    std_devs = np.zeros_like(n_list, dtype = np.float64)
    times = np.zeros_like(n_list, dtype = np.float64)

    for i in range(len(n_list)):
        start_time = time.time()
        n = n_list[i]
        
        if cv:
            cv_samples = control_variates(n)
            v[i] = cv_samples.mean()
            std_devs[i] = cv_samples.std() / np.sqrt(n)
        else:
            v[i], std_devs[i] = run_monte_carlo(S0, K, r, sigma, T, N, base**n, 
                                                payoff_function = payoff_function, method = method,
                                                sample_method = sample_method, antithetic = antithetic)

        times[i] = time.time() - start_time
        
    return (v, std_devs, times)

def format_data(data):
    df = pd.DataFrame(np.array(data).T, columns = {"Option Price (V)", "MOE", "Runtime"})
    df.index = 2 ** (df.index + 3)
    df.index.name = "Num Samples"
        
    return df
    


In [570]:
gbm_base = run_experiment(GBM, payoff_function = payoff_asian, num_trials = 15, base = 2)
format_data(gbm_base)

,Option Price (V),MOE,Runtime
Num Samples,,,
8,1.7410,0.6569,0.0034
16,3.6387,0.8432,0.0090
32,3.4565,0.7297,0.0213
64,3.3740,0.4871,0.0621
128,3.9189,0.4111,0.0890
256,3.8639,0.2492,0.1894
512,3.6486,0.1891,0.5076
1024,3.8474,0.1368,0.9136
2048,3.6570,0.0945,1.5517


In [566]:
antithetic_GBM = run_experiment(GBM, payoff_function = payoff_asian, antithetic = True, num_trials = 15, base = 2)
format_data(antithetic_GBM)

,Option Price (V),MOE,Runtime
Num Samples,,,
8,3.3192,0.9095,0.0104
16,4.0174,0.8244,0.0178
32,3.7463,0.5541,0.0339
64,3.5444,0.3576,0.1013
128,4.0132,0.2957,0.1144
256,3.6213,0.1826,0.3377
512,3.5807,0.1304,0.6632
1024,3.5393,0.0900,1.1120
2048,3.5656,0.0641,2.1924


In [564]:
sobol_gbm = run_experiment(GBM, payoff_function = payoff_asian, sample_method = quasi_Sobol, num_trials = 15, base = 2)
format_data(sobol_gbm)

,Option Price (V),MOE,Runtime
Num Samples,,,
8,3.5104,1.3945,0.0292
16,4.2905,1.0441,0.0321
32,3.1922,0.6305,0.0424
64,3.4595,0.4503,0.0652
128,3.7322,0.3832,0.0689
256,3.9007,0.2823,0.1911
512,3.6383,0.1908,0.4118
1024,3.6340,0.1273,0.6125
2048,3.6196,0.0907,1.2506


In [557]:
control_variates_geom = run_experiment(cv = True, num_trials = 15, base = 2)
format_data(control_variates_geom)

,Option Price (V),MOE,Runtime
Num Samples,,,
8,3.6264,0.0741,0.0531
16,3.6427,0.0574,0.0524
32,3.6525,0.0467,0.0979
64,3.6700,0.0421,0.0650
128,3.6508,0.0402,0.0367
256,3.6559,0.0355,0.0845
512,3.6574,0.0316,0.0612
1024,3.6608,0.0286,0.0845
2048,3.6695,0.0273,0.1154
